Phaser
===

In [ ]:
from tkinter import Tk, Canvas
from random import randrange
from turtle import Vec2D as vect

MAX_X=MAX_Y=600

root=Tk()
cnv=Canvas(root, width=MAX_X, height=MAX_Y, bg="gray")
cnv.pack()

# La longueur de chaque flèche
L=60

# les N extrémités des flèches
N=6
points=[(randrange(MAX_X),randrange(MAX_Y)) for _ in range(N)]



def origin(B, C, L):
    """
    Calcule l'origine A de la flèche connaissant :
    - l'extrémité B de la flèche
    - un point C visé par la flèche
    - la longueur L de la flèche
    """
    v=vect(*C)-vect(*B)
    k=-L/abs(v)
    A=k*v+vect(*B)
    return A

    
def shot(C):
    # C = poit de convergence des flèches
    # Dessine les flèches et les pointillés
    # depuis chaque point de la liste points
    for B in points:
        A=origin(B, C, L)
        # Créatopn de la flèche
        cnv.create_line(A,B, width=7, fill="green yellow", arrow='last', arrowshape=(18,30, 8))
        # Création des pointillés
        cnv.create_line(B,C, fill="green yellow", dash=3)    
        
def g(event):
    cnv.delete('all')
    x=event.x
    y=event.y
    shot((x,y))



root.bind("<Motion>",g)
    
        
        
# Dessin statique : la cible des flèches est un point aléatoire
shot((randrange(MAX_X),randrange(MAX_X)))


root.mainloop() 


Feu de foret
==

In [32]:
from random import sample, randrange
from tkinter import *

COLORS=["ivory", "lime green", "red", "gray75"]

def random_forest(p, n):
    # génération aléatoire des arbres dans la parcelle
    units=[(line,col) for col in range(n) for line in range(n)]
    ntrees=int(n**2*p)
    trees=sample(units,ntrees)
    states=[[0]*n for _ in range(n)]
    for (i,j) in trees:
        states[i][j]=1    
    return states, trees

def voisins(n, i, j):
    """Dans une grille n x n, indices (ligne, colonne) des voisins 
    dans la grille d'un sommet (i,j) de la grille"""
    return [(a,b) for (a, b) in [(i, j+1),(i, j-1), (i-1, j), (i+1,j)] if a in range(n) and b in range(n)]

def update_states(states, fires):
    # Modèle
    # Modifie le plateau en son nouvel état
    # renvoie les positions des arbes en feu
    n=len(states)
    to_fire=[]
    for (line, col) in fires:
        for (i, j) in voisins(n, line, col):
            if states[i][j]==1:
                to_fire.append((i, j))
    for (line, col) in to_fire:
        if states[line][col]==1:
            states[line][col]=2
    for (line, col) in fires:
        states[line][col]=3
    
    return list(set(to_fire)) 
                
def fill(cnv, states, unit):
    # Vue Tkinter
    # dessine sol et arbres
    n=len(states)    
    for line in range(n):
        for col in range(n):
            A=(unit*col, unit*line)
            B=(unit*(col+1), unit*(line+1))
            state=states[line][col]
            color=COLORS[state]
            cnv.create_rectangle(A, B, fill=color, outline='')

def start_fire(states, trees):
    # Modèle
    "Met le feu aléatoirement (code = 2) à l'arbre en position (i,j)"
    i,j= trees[randrange(len(trees))]
    states[i][j]=2
    return (i,j)

def set_fire(event):
    # La fonction associée au clic de souris
    # pour mettre le feu
    # A écrire par vous-même
    line=event.x//10
    col=event.y//10
    if (states[col][line]==1):
        states[col][line]=2
        fires.append((col,line))
    pass
    
def propagate():
    # Vue Tkinter
    # Animation du feu
    global fires, nfires
    fires=update_states(states, fires)
    k=len(fires)
    nfires+=k
    cnv.delete("all")
    fill(cnv, states, unit)
    cnv.after(200, propagate)
    if k==0:
        return

p=0.60
n=50
unit=10
nfires=0

root = Tk()
side=unit*n

cnv = Canvas(root, width=unit*n, height=unit*n, background="ivory") 
cnv.pack(padx=10, pady=10, side=LEFT)

states, trees=random_forest(p, n)

i, j=start_fire(states, trees)
fires=[(i,j)]  

# on remplit la parcelle
fill(cnv, states, unit)

# on anime
propagate()
cnv.bind("<Button-1>", set_fire)

root.mainloop()
